Source: 
<ol>
<li><a href="http://pgbovine.net/cpython-internals.htm">CPython internals: A ten-hour codewalk through the Python interpreter source code</a></li>
<li><a href="http://www.aosabook.org/en/500L/a-python-interpreter-written-in-python.html">A Python Interpreter Written in Python</a></li>
</ol>

# Building an Interpreter

## A Tiny Interpreter

In [49]:
class Interpreter(object):
    def __init__(self):
        self.stack = []

    def LOAD_VALUE(self, number):
        self.stack.append(number)

    def PRINT_ANSWER(self):
        answer = self.stack.pop()
        print(answer)

    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)

    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        numbers = what_to_execute["numbers"]
        for each_step in instructions:
            instruction, argument = each_step
            if instruction == "LOAD_VALUE":
                number = numbers[argument]
                self.LOAD_VALUE(number)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()

In [50]:
myInterpreter1 = Interpreter()

<img src="http://www.aosabook.org/en/500L/interpreter-images/interpreter-stack.png"/>

In [51]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),  # the first number
                     ("LOAD_VALUE", 1),  # the second number
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5] }

myInterpreter1.run_code(what_to_execute)

12


In [52]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                     ("LOAD_VALUE", 1),
                     ("ADD_TWO_VALUES", None),
                     ("LOAD_VALUE", 2),
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5, 8] }

myInterpreter1.run_code(what_to_execute)

20


## Variables

In [69]:
class Interpreter2(Interpreter):
    def __init__(self):
        Interpreter.__init__(self)
        self.environment = {}

    def __call__(self, *pargs, **kwargs):
        self.execute(*pargs, **kwargs)
        
    def STORE_NAME(self, name):
        val = self.stack.pop()
        self.environment[name] = val

    def LOAD_NAME(self, name):
        val = self.environment[name]
        self.stack.append(val)

    def parse_argument(self, instruction, argument, what_to_execute):
        """ Understand what the argument to each instruction means."""
        numbers = ["LOAD_VALUE"]
        names = ["LOAD_NAME", "STORE_NAME"]

        if instruction in numbers:
            argument = what_to_execute["numbers"][argument]
        elif instruction in names:
            argument = what_to_execute["names"][argument]

        return argument

    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)

            if instruction == "LOAD_VALUE":
                self.LOAD_VALUE(argument)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()
            elif instruction == "STORE_NAME":
                self.STORE_NAME(argument)
            elif instruction == "LOAD_NAME":
                self.LOAD_NAME(argument)
    
    # Implementation of run_code using
    # Python's dynamic method lookup
    def execute(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)
            bytecode_method = getattr(self, instruction)
            if argument is None:
                bytecode_method()
            else:
                bytecode_method(argument)                

In [70]:
myInterpreter2 = Interpreter2()

In [71]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                     ("STORE_NAME", 0),
                     ("LOAD_VALUE", 1),
                     ("STORE_NAME", 1),
                     ("LOAD_NAME", 0),
                     ("LOAD_NAME", 1),
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [1, 2],
    "names":   ["a", "b"] }

myInterpreter2.run_code(what_to_execute)

3


In [72]:
myInterpreter2(what_to_execute)

3


# Real Python Bytecode

In [163]:
from dis import dis, opname

In [82]:
def cond():
    x = 3
    if x < 5:
        return 'yes'
    else:
        return 'no'

<code>cond.\__code__</code>  is the code object associated to <code>cond</code>

In [112]:
cond.__code__

<code object cond at 0x180fa91300, file "<ipython-input-82-322d0ec9f2b5>", line 1>

<code>cond.\__code__.co_code</code>  is the bytecode

In [113]:
cond.__code__.co_code  # the bytecode as raw bytes

b'd\x01}\x00|\x00d\x02k\x00r\x10d\x03S\x00d\x04S\x00d\x00S\x00'

In [168]:
print(list(cond.__code__.co_code))  # the bytecode as numbers

[100, 1, 125, 0, 124, 0, 100, 2, 107, 0, 114, 16, 100, 3, 83, 0, 100, 4, 83, 0, 100, 0, 83, 0]


In [169]:
list(cond.__code__.co_code)[::2]

[100, 125, 124, 100, 107, 114, 100, 83, 100, 83, 100, 83]

<a href="https://github.com/python/cpython/blob/master/Include/opcode.h">Include/opcode.h</a> contains

In [170]:
import urllib3
urllib3.disable_warnings()
http = urllib3.PoolManager()
r = http.request('GET', 'https://raw.githubusercontent.com/python/cpython/master/Include/opcode.h')

In [171]:
for _l in r.data.decode().split("\n"):
    _split = _l.strip().split()
    try: 
        if _split and int(_split[-1]) in list(cond.__code__.co_code)[::2]:
            print(f"{_split[2]:3} : {_split[1]}")
    except ValueError:
        pass

83  : RETURN_VALUE
100 : LOAD_CONST
107 : COMPARE_OP
114 : POP_JUMP_IF_FALSE
124 : LOAD_FAST
125 : STORE_FAST


In [172]:
dis(cond)

  2           0 LOAD_CONST               1 (3)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_CONST               2 (5)
              8 COMPARE_OP               0 (<)
             10 POP_JUMP_IF_FALSE       16

  4          12 LOAD_CONST               3 ('yes')
             14 RETURN_VALUE

  6     >>   16 LOAD_CONST               4 ('no')
             18 RETURN_VALUE
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [181]:
set(list(cond.__code__.co_code)[::2])

{83, 100, 107, 114, 124, 125}

In [182]:
for _c in set(sorted(list(cond.__code__.co_code)[::2])):
    print(f"{_c:3} : {opname[_c]}")

100 : LOAD_CONST
107 : COMPARE_OP
114 : POP_JUMP_IF_FALSE
 83 : RETURN_VALUE
124 : LOAD_FAST
125 : STORE_FAST


In [185]:
list(set(list(cond.__code__.co_code)[::2]))

[100, 107, 114, 83, 124, 125]